In [1]:
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [2]:
def augmented(D):
    n = len(D)
    a0 = np.array([np.ones(n)]).T
    aug_D = np.hstack((D,a0))
    return aug_D

## Linear Kernel

Calculates the dot product of all pairwise points ($\phi_i$) in dataset D and returns that matrix

$$K_{i,j} = K(\phi_i, \phi_j) $$


$$K(\phi_i, \phi_j) = \phi_{i}^T \cdot \phi_j$$

In [3]:
def linearKernel(D):
    n = len(D)
    K = np.zeros((n, n))
    
    for i in range (n):
        for j in range (n):
            x = D[i,:]
            y = D[j,:]
            K[i][j] = np.dot(x,y)
            
    return K

In [4]:
def linearKernelpoint(D,z):
    n = len(D)
    K = np.zeros(n)
    
    for i in range(n):
        x = D[i,:]
        K[i] = np.dot(x, z)
        
    return K


## Inhomogenous Quadratic Kernel

$$K_{i,j} = K(\phi_i, \phi_j) $$


$$ K(\phi_i, \phi_j)  = (1 + \phi_{i}^T \cdot \phi_j)^2$$

In [5]:
def quadKernel(D):
    n = len(D)
    K = np.zeros((n, n))
    
    for i in range (n):
        for j in range (n):
            x = D[i,:]
            y = D[j,:]
            K[i][j] = (1 + np.dot(x,y))**2
            
    return K

In [6]:
def quadKernelpoint(D,z):
    n = len(D)
    K = np.zeros(n)
    
    for i in range(n):
        x = D[i,:]
        K[i] = (1 + np.dot(x,z))**2
        
    return K


## Gaussian Kernel

$$K_{i,j} = K(\phi_i, \phi_j) $$


$$ K(\phi_i, \phi_j)  =  exp(- \frac{\phi_{i}^T \cdot \phi_j}{2\sigma^2})$$

In [7]:
def gaussianKernel(D, sigma):
    n = len(D)
    K = np.zeros((n, n))
    
    for i in range (n):
        
        for j in range (n):
            
            x = D[i,:]
            y = D[j,:]
            diff = x - y
            K[i][j] = np.exp(- np.dot(diff,diff) / (2*sigma**2))
            
    return K

In [8]:
def gaussianKernelpoint(D,sigma, z):
    n = len(D)
    K = np.zeros(n)
    
    for i in range(n):
        
        x = D[i,:]
        diff = x - z
        K[i] = np.exp(-np.dot(diff,diff) / (2*sigma**2))
        
    return K

## Dual SVM Algorithm

![](svmalgo.png)


In [9]:
def dualSVM(D, y, kernel_type, C, eps, spread = 1):
    
    D_aug = augmented(D)
    y = y[:,0]
    
    #kernel matrix creation
    if(kernel_type == "linear"):
        K = linearKernel(D_aug)
    elif(kernel_type == "quadratic"):
        K = quadKernel(D_aug)
    elif(kernel_type == "gaussian"):
        K = gaussianKernel(D_aug, spread)
    
    
    n = len(K)
    eta = []
    for i in range(n):
        eta.append( 1 / K[i][i] )
        
        
    t = 0
    alpha_0 = np.zeros(n)
    alpha_prev = alpha_0
    alpha_curr = alpha_0
    
    while True:
        
        alpha_prev = alpha_curr
        alpha = alpha_curr
        
        
        for k in range(n):
            
            summation_term = 0.0
            
            for i in range(n):
                summation_term = summation_term +  alpha[i] * y[i] * K[i][k]
            
            alpha[k] = alpha[k] + eta[k] * (1 - y[k] * summation_term)
    
            if(alpha[k] < 0):
                 alpha[k] = 0
            if(alpha[k] > C):
                alpha[k] = C
        
        alpha_curr = alpha
        diff = alpha_curr - alpha_prev
        
        if(np.dot(diff, diff)**.5 < eps):
            return np.array([alpha_curr]).T
        

        

In [10]:
def testing(Z, D, y, alpha, process, sigma = 1):
    Z_aug = augmented(Z)
    D_aug = augmented(D)
    n = len(Z)
    y_hat = np.zeros(n)
    alpha = alpha[:,0]
    y = y[:,0]
    m = len(D)
    
    for i in range (n):
        
        y_i = 0
        z = Z_aug[i,:]
        
        if(process == "linear"):
            K = linearKernelpoint(D_aug, z)
        elif(process == "quadratic"):
            K = quadKernelpoint(D_aug, z)
        else:
            K = gaussianKernelpoint(D_aug, sigma, z) 
        
        
        for j in range (m):
            y_i = y_i + alpha[j] * y[j] * K[j]
        
            if(y_i >= 0):
                y_hat[i] = 1
            else:
                y_hat[i] = -1
            
    return np.array([y_hat]).T 

In [11]:
def supportVectors(alpha):
    alpha = alpha[:,0]
    n = len(alpha)
    print("Support Vector Indices...")
    for i in range(n):
        if(alpha[i] > 0):
            print(i)
    
    print("End of SV List")
    return 

In [12]:
def accuracy(y, y_hat):
    y = y[:,0]
    y_hat = y_hat[:,0]
    count = 0
    for i in range(len(y)):
        if(y[i] == y_hat[i]):
            count = count + 1
    
    return count/len(y)

In [13]:
def weight(D, y, alpha):
    D_aug = augmented(D)
    alpha = alpha[:,0]
    y = y[:,0]
    w = 0
    n = len(D)
    
    for i in range(n):
        x = D[i,:]
        w = w + alpha[i] * y[i] * x
        
    return w
    
        

## Training Set

In [14]:
train = np.genfromtxt("Concrete_Data_RNorm_Class_train.txt", delimiter=',')

d = len(train[0])
D_train = train[:,np.arange(d-1)]
y_train = np.array([train[:,d-1]]).T

alpha = dualSVM(D_train, y_train, "gaussian", 3, .001, .6)
w = weight(D_train, y_train, alpha)
print(w)
supportVectors(alpha)

[ 4.76273706 -1.59028586 -4.26308975 -7.4058505  -0.0655426  -2.46506257
 -4.11070675  5.82118815]
Support Vector Indices...
0
1
2
3
4
5
6
7
8
9
10
11
13
15
16
17
18
19
20
21
22
23
25
26
27
28
29
30
31
32
33
34
35
36
37
41
43
45
46
47
48
51
52
53
56
58
59
60
61
62
64
67
68
69
70
72
74
75
79
81
82
84
87
89
90
91
93
96
99
101
102
104
105
106
107
109
110
111
112
113
114
116
117
118
121
122
124
125
127
129
130
131
132
133
135
136
137
139
140
142
144
146
147
148
149
151
152
154
158
162
163
164
165
167
169
170
174
175
180
181
183
186
187
188
189
192
194
195
196
198
203
204
209
210
212
219
220
223
224
225
227
229
231
232
233
234
238
242
243
244
245
250
251
253
254
256
258
260
261
262
263
264
267
268
269
272
274
275
277
280
281
283
284
287
289
290
291
292
294
295
297
299
300
301
302
303
304
305
306
307
309
310
311
312
313
314
316
321
322
323
324
327
328
329
330
332
334
335
336
338
339
341
342
343
344
345
346
348
349
350
351
354
360
362
363
364
365
367
369
372
373
375
380
384
388
389
391
392
39

## Testing Set

In [152]:
test = np.genfromtxt("Concrete_Data_RNorm_Class_test.txt", delimiter=',')
    
d = len(test[0])
D_test = test[:,np.arange(d-1)]
y_test = np.array([test[:,d-1]]).T
y_hat = testing(D_test, D_train, y_train, alpha, "gaussian", .6)
acc = accuracy(y_test, y_hat)
print("Testing Accuracy: ", acc)

Testing Accuracy:  0.9077669902912622


### The best combination was a Gaussian Kernel with C = 3, eps = .001, and spread = .6.  This netted a Testing Accuracy:  90.77%